In [1]:
import requests
import sqlite3
from config import SEATGEEK_CLIENT_ID, SEATGEEK_CLIENT_SECRET

In [22]:
def fetch_events():
    api_url = 'https://api.seatgeek.com/2/events'
    params = {
        'client_id': SEATGEEK_CLIENT_ID,
        'taxonomies.name': 'concert',  # Filter for music concerts
        'per_page': 500,  # Fetch top 500 events
        'sort': 'score.desc'  # Sort by popularity score, descending
    }

    response = requests.get(api_url, params=params)
    if response.status_code == 200:
        return response.json()['events']
    else:
        print(f"Failed to fetch events: {response.status_code}, Response: {response.text}")
        return None

In [4]:
# Parse API Response
def parse_event_data(events):
    parsed_data = []
    for event in events:
        event_name = event['title']
        event_date = event['datetime_local']
        venue_name = event['venue']['name']
        city_name = event['venue']['city']
        lowest_price = event['stats'].get('lowest_price', 'N/A')
        highest_price = event['stats'].get('highest_price', 'N/A')

        parsed_data.append({
            'event_name': event_name,
            'event_date': event_date,
            'venue_name': venue_name,
            'city_name': city_name,
            'lowest_price': lowest_price,
            'highest_price': highest_price
        })

    return parsed_data

In [5]:
def create_database():
    conn = sqlite3.connect('events.db')  # This creates a file 'events.db' in the current directory
    c = conn.cursor()

    # SQL statement to create a table
    c.execute('''
        CREATE TABLE IF NOT EXISTS events (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            event_name TEXT,
            event_date TEXT,
            venue_name TEXT,
            city_name TEXT,
            lowest_price REAL,
            highest_price REAL
        )
    ''')

    conn.commit()  # Commit the changes
    conn.close()   # Close the connection to the database



In [6]:
def insert_data(parsed_data):
    conn = sqlite3.connect('events.db')
    c = conn.cursor()

    for event in parsed_data:
        # Insert each event into the events table
        c.execute('''
            INSERT INTO events (event_name, event_date, venue_name, city_name, lowest_price, highest_price)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (event['event_name'], event['event_date'], event['venue_name'], event['city_name'], event['lowest_price'], event['highest_price']))

    conn.commit()
    conn.close()

In [25]:
def clear_data():
    conn = sqlite3.connect('events.db')
    c = conn.cursor()
    c.execute('DELETE FROM events')
    c.execute('UPDATE sqlite_sequence SET seq = 0 WHERE name = "events"')  # Reset auto-increment counter
    conn.commit()
    conn.close()

def main():
    create_database()
    try:
        clear_data()  # Attempt to clear the data
        print("Data cleared successfully.")
    except Exception as e:
        print(f"Error clearing data: {e}")

    events = fetch_events()
    if events:
        parsed_data = parse_event_data(events)
        insert_data(parsed_data)

        top_25_query = "SELECT * FROM events ORDER BY id LIMIT 25;"
        top_25_events = run_query(top_25_query)
        for event in top_25_events:
            print(event)

if __name__ == '__main__':
    main()

Data cleared successfully.
(1, 'Drake with J. Cole', '2024-01-26T20:00:00', 'Paycom Center', 'Oklahoma City', 173.0, 21624.0)
(2, 'Billy Joel', '2023-11-22T20:00:00', 'Madison Square Garden', 'New York', 171.0, 12026.0)
(3, 'Rod Wave - Nostalgia Tour', '2023-12-17T20:00:00', 'Amerant Bank Arena', 'Sunrise', 176.0, 1066.0)
(4, 'Billy Joel & Stevie Nicks', '2024-03-09T19:00:00', 'AT&T Stadium', 'Arlington', 135.0, 2672.0)
(5, 'Trans-Siberian Orchestra-The Ghosts Of Christmas Eve-Matinee', '2023-11-21T15:00:00', 'Delta Center', 'Salt Lake City', 68.0, 144.0)
(6, 'Taylor Swift', '2024-10-27T19:00:00', 'Caesars Superdome', 'New Orleans', 1095.0, 76639.0)
(7, 'Taylor Swift', '2024-10-20T19:00:00', 'Hard Rock Stadium', 'Miami Gardens', 1455.0, 84164.0)
(8, 'Travis Scott', '2023-11-21T19:00:00', 'Moody Center ATX', 'Austin', 74.0, 11664.0)
(9, 'Trans-Siberian Orchestra-The Ghosts Of Christmas Eve', '2023-11-21T19:00:00', 'Delta Center', 'Salt Lake City', 81.0, 144.0)
(10, 'Drake with J. Cole',

In [26]:
def update_event_prices(event):
    conn = sqlite3.connect('events.db')
    c = conn.cursor()
    c.execute('''
        UPDATE events 
        SET lowest_price = ?, highest_price = ? 
        WHERE event_id = ?
    ''', (event['lowest_price'], event['highest_price'], event['event_id']))
    conn.commit()
    conn.close()

def update_data(parsed_data):
    for event in parsed_data:
        update_event_prices(event)

In [ ]:
import datetime

def delete_past_events():
    today = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    conn = sqlite3.connect('events.db')
    c = conn.cursor()
    c.execute('''
        DELETE FROM events 
        WHERE event_date < ?
    ''', (today,))
    conn.commit()
    conn.close()

# Call this function as needed, possibly in the same interval as your updates